In [158]:
chars = set()
with open("names.txt", "r") as f:
    names = f.read().splitlines()
for name in names:
    chars.update(list(name))
chars = sorted(chars)
chars = ['.'] + list(chars) 
print(chars)


['.', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [159]:
stoi = {}
idx = 0
for i in range(len(chars)):
    stoi[chars[i]] = idx 
    idx+=1
for char_1 in chars:
    for char_2 in chars:
        stoi[(char_1, char_2)] = idx
        idx  +=1

itos = {i:c for c,i in stoi.items()}
print(stoi)

print(itos)


{'.': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26, ('.', '.'): 27, ('.', 'a'): 28, ('.', 'b'): 29, ('.', 'c'): 30, ('.', 'd'): 31, ('.', 'e'): 32, ('.', 'f'): 33, ('.', 'g'): 34, ('.', 'h'): 35, ('.', 'i'): 36, ('.', 'j'): 37, ('.', 'k'): 38, ('.', 'l'): 39, ('.', 'm'): 40, ('.', 'n'): 41, ('.', 'o'): 42, ('.', 'p'): 43, ('.', 'q'): 44, ('.', 'r'): 45, ('.', 's'): 46, ('.', 't'): 47, ('.', 'u'): 48, ('.', 'v'): 49, ('.', 'w'): 50, ('.', 'x'): 51, ('.', 'y'): 52, ('.', 'z'): 53, ('a', '.'): 54, ('a', 'a'): 55, ('a', 'b'): 56, ('a', 'c'): 57, ('a', 'd'): 58, ('a', 'e'): 59, ('a', 'f'): 60, ('a', 'g'): 61, ('a', 'h'): 62, ('a', 'i'): 63, ('a', 'j'): 64, ('a', 'k'): 65, ('a', 'l'): 66, ('a', 'm'): 67, ('a', 'n'): 68, ('a', 'o'): 69, ('a', 'p'): 70, ('a', 'q'): 71, ('a', 'r'): 72, ('a', 's'): 73, ('a', 't'): 74

In [160]:
import torch 
trigram_tensor = torch.zeros((812,28), dtype=torch.int32)

for name in names:
    chrs = ['.', '.'] + list(name) + ['.']

    for i in range(len(chrs) - 2):
        trigram_tensor[stoi[(chrs[i], chrs[i+1])], stoi[chrs[i+2]]] += 1
  

In [161]:
trigram_tensor = trigram_tensor.float()
trigram_tensor /=  trigram_tensor.sum(dim = 1, keepdim =  True)




In [162]:

def sample_from_trigram(trigram_tensor):
    """
    implementation of sampling a value from cumulative probability distribution
    """
    cdf_tensor = trigram_tensor.cumsum(0)
    rand_value = torch.rand(1)
    idx = 0
    for i in range(cdf_tensor.shape[0]):
        if cdf_tensor[i] > rand_value:
            idx = i
            break

    return idx

for i in range(200):
    idx = stoi[('.', '.')]
    word = ['.', '.']
    while True:
        next_idx = sample_from_trigram(trigram_tensor[idx, :])
        next_char = itos[next_idx]

        word.append(next_char)

        idx = stoi[(word[-2], word[-1])]
        if next_idx == 0:
            break
    print("".join(word[2:-1]))

oaqwentiliela
azire
kain
bissayosa
avy
saamichu
keidgen
emi
taihamesulia
sanne
naulietren
na
yah
da
em
kariscoria
kasiazhaa
maralet
amord
ellahie
carie
lya
rogio
adain
precius
jus
sakolby
len
jadamana
zubrian
xyon
amir
annajoua
hamona
tarjul
elievalayah
nash
bett
gran
crakeyshundanna
dia
amersyri
codierieliann
shaelzaya
ly
ix
race
aleishubri
aaleana
fark
dee
brya
aslin
el
renni
hia
novorane
mice
kin
keightlyndavici
fair
elamir
mer
zakah
nalmara
arleigh
que
dra
rajookeiddhwan
ros
safina
ledis
azion
akal
ivis
an
karri
ki
vinoramiraniamira
ston
oldeen
havin
jah
zymissamarlidyanimokuntzavyn
ny
gleya
srenesleth
macia
hangs
im
yah
siandriseo
omyah
arlo
de
zaynn
nyrie
tinlee
jah
aneyas
roserleydaleen
li
dah
reighaun
yux
henn
ke
kae
bret
de
aylasmili
trilow
han
erres
deani
mehashi
denna
pelari
ha
jamerwyn
da
jayahmarseno
cy
arraxeni
laiya
no
ty
aree
talidhyron
ann
te
ivoron
in
kori
adia
ganne
ron
jhes
aden
za
la
chmicksi
rin
chail
ty
ham
kairozeryssa
clyn
tai
kam
sari
sa
arhyra
kimeel
deya
fir

In [163]:
chars = set()
with open("names.txt", "r") as f:
    names = f.read().splitlines()
for name in names:
    chars.update(list(name))
chars = sorted(chars)
chars = ['.'] + list(chars) 
stoi = {}
idx = 0
for i in range(len(chars)):
    stoi[chars[i]] = idx 
    idx+=1

itos = {i:c for c,i in stoi.items()}
print(stoi)

print(itos)

{'.': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}
{0: '.', 1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z'}


In [164]:
import torch.nn.functional as F
import torch
x_feats = []
y_feats = []


for name in names:
    chrs = ['.'] + list(name) + ['.']
    for i in range(len(chrs) - 1):
        x_idx = torch.tensor(stoi[chrs[i]])
        y_idx = torch.tensor(stoi[chrs[i+1]])

        x_feats.append(x_idx)
        y_feats.append(y_idx)
        # print(x_feat, y_feat)
        # print(chrs[i], chrs[i+1])
x_feats = torch.tensor(x_feats)
y_feats = torch.tensor(y_feats)

x_feats = F.one_hot(x_feats, num_classes=len(chars))

x_feats = x_feats.float()

print(x_feats.shape)
print(y_feats.shape)

torch.Size([228146, 27])
torch.Size([228146])


In [165]:
# x_feats = (228146, 27)
# W = (27, 27)
# X @ W = (228146, 27) @ (27, 27) = (228146, 27)
W =  torch.randn((27, 27), requires_grad=True)
O = x_feats @ W
print(O.shape)

torch.Size([228146, 27])


In [166]:
print(O[0,:])

tensor([ 0.5710, -0.7529,  1.2221, -0.8897,  0.4779,  1.5621, -2.6768,  1.6389,
        -0.3850, -0.5993, -0.8206, -0.7365,  0.5799,  0.2100, -0.5465, -0.7475,
         0.5449,  0.4951, -2.2089, -1.8854,  0.1375, -0.2478,  0.3584,  1.6314,
         1.2320,  2.6693,  0.0539], grad_fn=<SliceBackward0>)


In [167]:

print(O[0,:])
O = O.exp()
O_sum = O.sum(1, keepdim=True)
O_softmax = O / O_sum
print(O_sum[0,:])
print(O_softmax.shape)
print(O_softmax[0,:])

tensor([ 0.5710, -0.7529,  1.2221, -0.8897,  0.4779,  1.5621, -2.6768,  1.6389,
        -0.3850, -0.5993, -0.8206, -0.7365,  0.5799,  0.2100, -0.5465, -0.7475,
         0.5449,  0.4951, -2.2089, -1.8854,  0.1375, -0.2478,  0.3584,  1.6314,
         1.2320,  2.6693,  0.0539], grad_fn=<SliceBackward0>)
tensor([54.8763], grad_fn=<SliceBackward0>)
torch.Size([228146, 27])
tensor([0.0323, 0.0086, 0.0619, 0.0075, 0.0294, 0.0869, 0.0013, 0.0938, 0.0124,
        0.0100, 0.0080, 0.0087, 0.0325, 0.0225, 0.0106, 0.0086, 0.0314, 0.0299,
        0.0020, 0.0028, 0.0209, 0.0142, 0.0261, 0.0931, 0.0625, 0.2629, 0.0192],
       grad_fn=<SliceBackward0>)


In [168]:
print(O_softmax.shape)

torch.Size([228146, 27])


In [169]:
loss = O_softmax[ torch.arange(O_softmax.shape[0]), y_feats]
loss = -loss.log().mean()

In [ ]:
for i in range(500):
    O = x_feats @ W
    O = O.exp()
    O_sum = O.sum(1, keepdim=True)
    O_softmax = O / O_sum
    loss = O_softmax[ torch.arange(O_softmax.shape[0]), y_feats]
    loss = -loss.log().mean() + 0.01 * (W ** 2).mean() # adding regularization is same as adding a small initial value to bigram frequency matrix, it makes the model smoother
    loss.backward() # this only calculates the gradients, yu need to manually do w.data -= 10 * W.grad in order to update the weights 
    W.data -= 10 * W.grad
    W.grad.zero_() # this is important, otherwise the gradients will accumulate
    print(loss)




tensor(2.4900, grad_fn=<AddBackward0>)
tensor(2.4900, grad_fn=<AddBackward0>)
tensor(2.4900, grad_fn=<AddBackward0>)
tensor(2.4899, grad_fn=<AddBackward0>)
tensor(2.4899, grad_fn=<AddBackward0>)
tensor(2.4899, grad_fn=<AddBackward0>)
tensor(2.4898, grad_fn=<AddBackward0>)
tensor(2.4898, grad_fn=<AddBackward0>)
tensor(2.4898, grad_fn=<AddBackward0>)
tensor(2.4898, grad_fn=<AddBackward0>)
tensor(2.4897, grad_fn=<AddBackward0>)
tensor(2.4897, grad_fn=<AddBackward0>)
tensor(2.4897, grad_fn=<AddBackward0>)
tensor(2.4896, grad_fn=<AddBackward0>)
tensor(2.4896, grad_fn=<AddBackward0>)
tensor(2.4896, grad_fn=<AddBackward0>)
tensor(2.4895, grad_fn=<AddBackward0>)
tensor(2.4895, grad_fn=<AddBackward0>)
tensor(2.4895, grad_fn=<AddBackward0>)
tensor(2.4895, grad_fn=<AddBackward0>)
tensor(2.4894, grad_fn=<AddBackward0>)
tensor(2.4894, grad_fn=<AddBackward0>)
tensor(2.4894, grad_fn=<AddBackward0>)
tensor(2.4893, grad_fn=<AddBackward0>)
tensor(2.4893, grad_fn=<AddBackward0>)
tensor(2.4893, grad_fn=<A

In [171]:


for i in range(100): 
    word = []   
    idx = 0
    while True:
        x_feat = F.one_hot(torch.tensor(idx), num_classes=len(chars)).float()
        O = x_feat @ W  # O shape: [vocab_size]
        O = O.exp()
        O_sum = O.sum()  # scalar
        O_softmax = O / O_sum  # shape: [vocab_size]
        O_softmax = O_softmax.unsqueeze(0)  # make it 2D for torch.multinomial
        idx = torch.multinomial(O_softmax, 1).item()  # get integer index directly
        word.append(itos[idx])
        if idx == 0:
            break
    print("".join(word))

leock.
kilasanesh.
lelices.
emad.
ah.
jaylulust.
jaa.
jelerys.
prelirayahallolevigeydrh.
tes.
amaleya.
rendnah.
mashahw.
rielynorliahaccaren.
k.
jak.
jamara.
tviesajeenahauutfn.
jan.
ciannnasticisanakamesh.
rlaserren.
e.
vitrielavind.
brahniatan.
aahva.
haly.
endarsapverisleaky.
zabih.
la.
m.
maa.
dryvoanahn.
anighayan.
ali.
jureetaizahathaynpeedilesumeigebasibolllorn.
lavkallah.
son.
rayarran.
jixsa.
dr.
an.
en.
dzamabrettlilyitoje.
yanonajain.
jxzm.
tre.
elllelinyn.
apach.
ga.
maniashi.
coxdronylatetooolilil.
abr.
meei.
di.
jaceywy.
lbyad.
zalyl.
rin.
ko.
ai.
gzusavia.
ynistyiathe.
gzilelilmyniya.
kvoviyaleorlelami.
mlyempenddya.
melliselay.
ve.
danani.
hospayinaeirifgh.
rlwiah.
she.
cdy.
zarisaiarivinin.
joheshan.
bia.
grx.
cphr.
mwcah.
sor.
zia.
kammanaver.
siranaz.
jukh.
stakay.
ya.
an.
meluase.
garurtani.
vze.
lelat.
annaran.
onan.
k.
sa.
an.
omahen.
ddorn.
isicha.
eeiyala.
dreovikan.


In [172]:
print(W)

tensor([[-2.8541e+00,  1.8222e+00,  6.0388e-01,  7.7032e-01,  8.6212e-01,
          7.6314e-01, -5.4390e-01, -6.7240e-02,  2.0114e-01, -1.9186e-01,
          1.2225e+00,  1.4243e+00,  7.8962e-01,  1.2693e+00,  4.7289e-01,
         -6.0047e-01, -3.3036e-01, -1.9560e+00,  8.3143e-01,  1.0579e+00,
          6.0541e-01, -2.0857e+00, -6.4754e-01, -8.5202e-01, -1.6498e+00,
         -2.9200e-01,  2.6237e-01],
        [ 2.3353e+00, -1.4721e-01, -1.7464e-01, -3.1596e-01,  4.8229e-01,
          7.2233e-02, -1.5391e+00, -1.3371e+00,  1.2886e+00,  9.4242e-01,
         -1.2949e+00, -1.2577e-01,  1.3694e+00,  9.3267e-01,  2.1356e+00,
         -2.0880e+00, -1.9384e+00, -2.1924e+00,  1.6250e+00,  5.5279e-01,
          6.4964e-02, -5.2676e-01,  2.5927e-01, -1.4926e+00, -1.2721e+00,
          1.1596e+00, -3.9371e-01],
        [ 4.4943e-01,  2.1200e+00, -7.8971e-01, -4.9260e-01,  3.6238e-01,
          2.8832e+00, -6.1114e-01, -8.1363e-01,  1.8764e-01,  1.6472e+00,
         -8.7289e-01, -4.9853e-01,  6.83

chapter 2 - MLP ----